<a href="https://colab.research.google.com/github/a3mad/ml/blob/main/customer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
from models.financial_model import train_financial_model
from models.demographic_model import train_demographic_model
from models.behavior_model import train_behavior_model
from models.communication_model import train_communication_model
from models.customer_value_model import train_customer_value_model
from models.segmentation_model import train_segmentation_model
from models.marketing_model import train_marketing_model
from models.premium_model import train_premium_model
import pandas as pd



def main():
  file_path = '/content/customer_segmentation_data.csv'
  data = pd.read_csv(file_path)
  financial_model = train_financial_model(data)
  demographic_model = train_demographic_model(data)
  behavior_model = train_behavior_model(data)
  communication_model = train_communication_model(data)
  customer_value_model = train_customer_value_model(data)
  segmentation_model = train_segmentation_model(data)
  marketing_model = train_marketing_model(data)
  premium_model = train_premium_model(data)

  # Example of making predictions with one of the models
  predictions = financial_model.predict(data)
  print(predictions)

  #applying the model to data
  data['Financial Cluster'] = financial_model['clustering'].labels_
  data['Demographic Cluster'] = demographic_model['clustering'].labels_
  print(data[['Customer ID', 'Financial Cluster', 'Demographic Cluster']].head())

if __name__ == '__main__':
    main()



[0 0 1 ... 1 1 2]
   Customer ID  Financial Cluster  Demographic Cluster
0        84966                  0                    0
1        95568                  0                    0
2        10544                  1                    0
3        77033                  1                    0
4        88160                  0                    0
